In [ ]:
# Calculate supply (total per mill) = done
# Converted all deliveries at historic rate (in RUB) = done
# Check and remove outliers per destination
import seaborn as sns
import warnings
import numpy as np
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
%matplotlib inline
sns.set()
# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

In [ ]:
import geocoder
from pprint import pprint
key = '484ae73bfa5b43d28bde740af59c0c0c' # get api key from: https://opencagedata.com
mills = dict.fromkeys(demand['точка отгрузки'])
cities = dict.fromkeys(demand['город'])
for name in mills:
    result = geocoder.opencage(name, key=key)
    mills[name] = result.latlng
for name in cities:
    result = geocoder.opencage(name, key=key)
    cities[name] = result.latlng

In [ ]:
#Transform, rename, merge
mills_distance = pd.DataFrame.from_dict(mills, orient='index',columns=['lat_точка','long_точка'])
cities_distance = pd.DataFrame.from_dict(cities, orient='index',columns=['lat_город','long_город'])
mills_distance.index.name = 'точка отгрузки'
cities_distance.index.name = 'город'
demand = demand.merge(mills_distance, left_on='точка отгрузки', right_on='точка отгрузки')
demand = demand.merge(cities_distance, left_on='город', right_on='город')

In [ ]:
#Calculate distance between two points, calculate cost per km = proxy for cost minimization function
import h3
demand['расстояние'] = demand.apply(lambda row: h3.point_dist((row['lat_точка'], row['long_точка']), (row['lat_город'], row['long_город'])), axis=1)
demand['стоимость за километр'] = demand.предложение_RUB/demand.расстояние

In [ ]:
#issue with inf type of float, which json doesn't recognise
demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']] = demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']].astype(str)
demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']] = demand[["lat_точка", "long_точка", "lat_город", "long_город", 'расстояние','стоимость за километр']].astype(str)